In [6]:
"""
https://dotsnlines.tistory.com/894?category=1034801
위 블로그 소스 참고하여 내용 분석하고 소스코드를 다 고침
기존 코드는 너무 비효율적이라 시간도 많이 걸리고 excel 출력으로 구현되어 있어 빨리 확인하기 쉽지 않음
"""
from pykrx import stock
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import os

pd.set_option('mode.chained_assignment',  None)

def row_pbr_company(date, years):
    """낮은 PBR 회사 20개의 투자 수익률 계산
    Args:
        date: 투자음    
        years: 투자기간
    Returns: DataFrame
        ticker 종목명 종가 상장주식수 PBR X년후종가 X년후상장주식수 상장주식수변동 pbr_rank 수익 수익률 투자년도
    """ 
    # code와 ticker이름을 먼저 구한다.
    start_date = stock.get_nearest_business_day_in_a_week(date=date) # 영업일
    codes = stock.get_market_ticker_list(start_date)
    
    # 투자 시작 시점 하위 PBR의 종가, 주식수 및 주요지표(PBR) 조회
    start_df_f = stock.get_market_fundamental_by_ticker(start_date)
    start_df_f = start_df_f[start_df_f['PBR'] > 0].sort_values(by=['PBR']).head(20) # PBR 0이상 하위 20개
    start_df_f.insert(0, '종목명', np.nan)
    for ticker in start_df_f.index.values:
        start_df_f.at[ticker, '종목명'] = stock.get_market_ticker_name(ticker)
    start_df_c = stock.get_market_cap_by_ticker(start_date)

    # 투자 종료 시점 종가, 주식수, 주요지표(PBR) 조회
    end_date = (datetime.datetime.strptime(date, '%Y%m%d') + relativedelta(years=years)).strftime('%Y%m%d')
    end_date = stock.get_nearest_business_day_in_a_week(date=end_date) # 영업일
    end_df_c = stock.get_market_cap_by_ticker(end_date)

    # 시작 시점과 종료 시점 데이터를 머지
    result_df = pd.merge(start_df_f, start_df_c, left_index=True, right_index=True)
    result_df = pd.merge(result_df, end_df_c, left_index=True, right_index=True)
    
    # 수익, 수익률 등을 계산하여 추가
    result_df['상장주식수변동'] = result_df['상장주식수_y'] - result_df['상장주식수_x']
    result_df['수익'] = result_df['종가_y'] - result_df['종가_x']
    # 주식수 변동(감소)에 따라 수익 수치 보정. 근데 증가는?
    # TODO 상장주식수가 증감하는 방식이 다양한데, 이걸로 수익률을 어떻게 계산할 수 있나? 유/무상 증자 등
    profit_df = result_df['종가_y'] * (1 + result_df['상장주식수변동'] / result_df['상장주식수_x']) - result_df['종가_x']
    result_df['수익'].loc[result_df['상장주식수변동'] < 0] = profit_df
    result_df['투자년도_x'] = np.array([start_date] * len(result_df))
    result_df['투자년도_y'] = np.array([end_date] * len(result_df))
    result_df['수익률'] = (result_df['수익'] / result_df['종가_x'])
    result_df = pd.concat([result_df, result_df.agg({'수익':['sum', 'mean'], '수익률':['sum', 'mean']})])

    return result_df

test_result = row_pbr_company('20190104', 1)
print(test_result)


              종목명       BPS    PER   PBR       EPS   DIV     DPS     종가_x  \
071970     STX중공업   47208.0   0.02  0.11  264128.0  0.00     0.0   5210.0   
011160       두산건설    9944.0   0.00  0.15       0.0  0.00     0.0   1480.0   
002300       한국제지   99906.0   7.92  0.16    2040.0  1.86   300.0  16150.0   
021820       세원정공   42284.0   0.00  0.16       0.0  1.48   100.0   6740.0   
007860         서연   21594.0   0.00  0.17       0.0  2.72   100.0   3680.0   
200880       서연이화   24673.0   3.54  0.18    1264.0  3.35   150.0   4480.0   
000950         전방   88632.0   0.00  0.19       0.0  2.97   500.0  16850.0   
058650      세아홀딩스  466950.0   2.75  0.20   33612.0  2.16  2000.0  92500.0   
003030     세아제강지주  229348.0   9.43  0.20    4877.0  3.91  1800.0  46000.0   
005010        휴스틸   54480.0  21.39  0.21     533.0  4.39   500.0  11400.0   
005030       부산주공    2247.0   0.00  0.21       0.0  0.00     0.0    472.0   
010100      한국프랜지    6832.0   0.00  0.21       0.0  0.00     0.0   1425.0   